In [1]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

import numpy as np




In [7]:
# !pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.0.1-cp37-none-any.whl size=10032 sha256=444d9a35b93b71a507c7b5b38d9e91bbe29f2c018b903b7e2ce4e7994653e635
  Stored in directory: /Users/ieunpyo/Library/Caches/pip/wheels/1d/0d/3b/6b9d4477a34b3905f246ff4e7acf6aafd4cc9b77d473629b77
Successfully built bayesian-optimization


## Colab Setting

In [23]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [24]:
work_dir = '/content/drive/work/gimhae_fire/'

In [68]:

#[10, 14, 18, 23, 27, 32, 36, 41, 45, 50]
# n_estimators 은표 [10,50] / 창현 [14,45] / 지현 [18,27,41] / 성호 [23,32,36]
n_estimators_list = [10,50]  # 자기가 돌리기로 한 부분을 리스트 형식으로 넣어주세요

# n =5 회 반복으로 하겠습니다.
n=5

## Data Loading

In [71]:
Train_X = np.load(work_dir + 'Train_X.npy',allow_pickle=True)
Test_X = np.load(work_dir + 'Test_X.npy',allow_pickle=True)
Validation_X = np.load(work_dir + 'Validation_X.npy',allow_pickle=True)

Train_y = np.load(work_dir  + 'Train_y.npy',allow_pickle=True)
Validation_y = np.load(work_dir + 'Validation_y.npy',allow_pickle=True)



FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/work/gimhae_fire/Train_X.npy'

In [72]:
Train_X.shape

(59199, 317)

# HYPERPARAMETER TUNING

## initial search

#### n_estimators : Number of trees in random forest
#### max_features : The number of features to consider when looking for the best split
#### max_depth : Maximum number of levels in tree
#### min_samples_split : Minimum number of samples required to split a node
#### min_samples_leaf : Minimum number of samples required at each leaf node

In [55]:
# n_estimators : Number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 10, stop = 50, num = 10)]

# max_features : The number of features to consider when looking for the best split
max_features = [int(x) for x in np.linspace(start = 10, stop = 30, num = 5)]

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10,100, num = 10)]
max_depth.append(None)

# min_samples_leaf : Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 5, 10]


# Create the random grid
random_grid = {'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_leaf': min_samples_leaf,
                }




In [56]:
random_grid

{'max_features': [10, 15, 20, 25, 30],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'min_samples_leaf': [1, 2, 4, 5, 10]}

In [41]:
from sklearn.model_selection import ParameterGrid

In [60]:
list(ParameterGrid(random_grid))

[{'max_depth': 10, 'max_features': 10, 'min_samples_leaf': 1},
 {'max_depth': 10, 'max_features': 10, 'min_samples_leaf': 2},
 {'max_depth': 10, 'max_features': 10, 'min_samples_leaf': 4},
 {'max_depth': 10, 'max_features': 10, 'min_samples_leaf': 5},
 {'max_depth': 10, 'max_features': 10, 'min_samples_leaf': 10},
 {'max_depth': 10, 'max_features': 15, 'min_samples_leaf': 1},
 {'max_depth': 10, 'max_features': 15, 'min_samples_leaf': 2},
 {'max_depth': 10, 'max_features': 15, 'min_samples_leaf': 4},
 {'max_depth': 10, 'max_features': 15, 'min_samples_leaf': 5},
 {'max_depth': 10, 'max_features': 15, 'min_samples_leaf': 10},
 {'max_depth': 10, 'max_features': 20, 'min_samples_leaf': 1},
 {'max_depth': 10, 'max_features': 20, 'min_samples_leaf': 2},
 {'max_depth': 10, 'max_features': 20, 'min_samples_leaf': 4},
 {'max_depth': 10, 'max_features': 20, 'min_samples_leaf': 5},
 {'max_depth': 10, 'max_features': 20, 'min_samples_leaf': 10},
 {'max_depth': 10, 'max_features': 25, 'min_samples_

In [62]:
def hyper_param_f1_score(n,n_estimators, max_features, max_depth, min_samples_leaf) :
    rfc = RandomForestClassifier(n_estimators=n_estimators, max_features=max_features, 
                                 max_depth=max_depth, min_samples_leaf=min_samples_leaf,
                                 n_jobs=-1)
    f1_score_list=[]
    for i in range(n) :
        rfc.fit(Train_X,Train_y)
        f1_score_list.append(f1_score(rfc.predict(Validation_X),Validation_y))
    m = np.mean(f1_score_list)
    s = np.std(f1_score_list)
    return [n_estimators, max_features, max_depth, min_samples_leaf, n, m,s]

In [67]:
result= []
for n_estimators_temp in n_estimators_list : 
    for i in range( len(list(ParameterGrid(random_grid)) ) ) :
        min_samples_leaf_temp = ParameterGrid(random_grid)[i]['min_samples_leaf']
        max_features_temp = ParameterGrid(random_grid)[i]['max_features']
        max_depth_temp = ParameterGrid(random_grid)[i]['max_depth']
    
        result.append(hyper_param_f1_score(n,n_estimators_temp, max_features_temp, max_depth_temp, min_samples_leaf_temp))
    

NameError: name 'n_estimators_list' is not defined

## second search
##### hyperparameter 완전 결정

## predict on test_X & submission
##### 정해진 hyperparameter로 N번 fitting 반복 predict -> f-score 최고점된 model로 제출

In [30]:
import datetime

In [31]:
now = datetime.datetime.now().strftime('%H:%M:%S')

In [33]:
'a' +now

'a17:56:26'

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# n_estimators : Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 50, num = 10)]

# max_features : The number of features to consider when looking for the best split
max_features = [int(x) for x in np.linspace(start = 10, stop = 100, num = 10)]

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# min_samples_split : Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# min_samples_leaf : Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 5, 10]


# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}



In [8]:
#Import libraries
import pandas as pd
import numpy as np
from bayes_opt import BayesianOptimization
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [9]:
#Bayesian optimization
def bayesian_optimization(dataset, function, parameters):
   X_train, y_train, X_test, y_test = dataset
   n_iterations = 5
   gp_params = {"alpha": 1e-4}

   BO = BayesianOptimization(function, parameters)
   BO.maximize(n_iter=n_iterations, **gp_params)

   return BO.max

In [11]:
def rfc_optimization(cv_splits):
    def function(n_estimators, max_depth, min_samples_split):
        return cross_val_score(
               RandomForestClassifier(
                   n_estimators=int(max(n_estimators,0)),                                                               
                   max_depth=int(max(max_depth,1)),
                   min_samples_split=int(max(min_samples_split,2)), 
                   n_jobs=-1, 
                   random_state=42,   
                   class_weight="balanced"),  
               X=X_train, 
               y=y_train, 
               cv=cv_splits,
               scoring="roc_auc",
               n_jobs=-1).mean()

    parameters = {"n_estimators": (10, 1000),
                  "max_depth": (20, 150),
                  "min_samples_split": (2, 10)}
    
    return function, parameters

In [12]:
#Train model
def train(X_train, y_train, X_test, y_test, function, parameters):
    dataset = (X_train, y_train, X_test, y_test)
    cv_splits = 4
    
    best_solution = bayesian_optimization(dataset, function, parameters)      
    params = best_solution["params"]

    model = RandomForestClassifier(
             n_estimators=int(max(params["n_estimators"], 0)),
             max_depth=int(max(params["max_depth"], 1)),
             min_samples_split=int(max(params["min_samples_split"], 2)), 
             n_jobs=-1, 
             random_state=42,   
             class_weight="balanced")

    model.fit(X_train, y_train)
    
    return model

In [34]:
rfc = RandomForestClassifier(hyperparam - from grid)

f1_score_list =[]
for i in range( n) # n : 정해진 회수
    rfc.fit(Train_X,Train_y)
    f1_score_list.append(f1_score(rfc.predict(Validation_X),Validation_y))


    
    
    


